# Dataset

In [21]:
import sys, os
sys.path.append(os.pardir)
from datasets.mnist import load_mnist

(X_train, y_train), (X_test, y_test) = load_mnist(flatten=True, normalize=False)

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


In [22]:
X_train.shape

(60000, 784)

In [23]:
y_train.shape

(60000,)

In [24]:
X_test.shape

(10000, 784)

In [25]:
y_test.shape

(10000,)

이미지 출력

In [26]:
import numpy as np
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [27]:
img = X_train[0]
label = y_train[0]
print(label)

5


In [28]:
img.shape

(784,)

In [29]:
img = img.reshape(28, 28)
img.shape

(28, 28)

In [30]:
img_show(img)

# 신경망의 추론 처리

In [31]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # prevents overflow
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

In [64]:
import pickle

def get_data():
    (X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return X_test, y_test

def init_network():
    with open("sample_weight.pkl", "rb") as f:
        network = pickle.load(f)

    return network

def predict(network, x):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

In [46]:
x, t = get_data()
network = init_network()

정확도를 기반으로 평가

In [48]:
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 확률 최대치 원소 인덱스 획득
    if p == t[i]: # 만약 확률이 가장 높은 인덱스 (추론된 숫자)가 실제 label과 같다면
        accuracy_cnt += 1

print("Accuracy:", str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


각 이미지에서 특정 데이터가 나올 확률 계산 후 가장 큰 원소의 인덱스 반환

정규화: 데이터를 특정 범위로 변환하는 것
전처리: 신경망 입력 데이터에 특정 변환을 가하는 것

# 배치 처리

In [49]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network["W1"], network["W2"], network["W3"]

In [50]:
x.shape

(10000, 784)

In [57]:
assert(x.shape[1] == x[0].shape[0])
x[0].shape

(784,)

In [59]:
assert(x[0].shape[0] == W1.shape[0])
W1.shape

(784, 50)

In [62]:
assert(W1.shape[1] == W2.shape[0])
W2.shape

(50, 100)

In [63]:
assert(W2.shape[1] == W3.shape[0])
W3.shape

(100, 10)

컴퓨터는 배치 처리를 통해 다수의 이미지를 한 번에 계산하는 것에 훨씬 유리함
수치 계산 라이브러리 대부분이 큰 계산에 대해 효율적으로 처리할 수 있도록 최적화되어 있음
I/O 횟수가 줄어 보다 빠른 계산이 가능

In [67]:
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size): # batch_size step 단위로 반복
    x_batch = x[i:i + batch_size] # i 부터 i + batch_size 까지의 데이터
    y_batch = predict(network, x_batch) # 각 원소에서 첫 번째 차원을 축으로 최댓값 인덱스를 찾도록 한 것
    p = np.argmax(y_batch, axis=1) # 최댓값 index를 가져옴
    accuracy_cnt += np.sum(p == t[i:i + batch_size])

In [68]:
print("Accuracy:", str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352
